# Torch
## Check GPU¶

In [1]:
import torch
import sys
sys.path.append('..')
from torchlib.utils import list_device,set_device

list_device()

------------ List Devices ------------
Device 0 :
GeForce RTX 2060
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB

Device 1 :
TITAN Xp
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB



the rosdep view is empty: call 'sudo rosdep init' and 'rosdep update'
/home/seel/.local/lib/python3.6/site-packages/torch/cuda/memory.py:346: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)


## Set torch default parameters¶

In [2]:
set_device(1)
torch.set_default_dtype(torch.float32)
torch.set_printoptions(precision=4)
torch.backends.cudnn.benchmark = True
torch.set_printoptions(sci_mode=False)

Using Device 1 : TITAN Xp


# Set Arguments

In [3]:
import argparse
import sys
import os
import time
import pickle

parser = argparse.ArgumentParser()

'''Training Parameters'''
parser.add_argument('--batch_size', type=int, default=400, help='minibatch size')
parser.add_argument('--num_epochs', type=int, default=200, help='number of epochs')
parser.add_argument('--grad_clip', type=float, default=5., help='clip gradients at this value')
parser.add_argument('--learning_rate', type=float, default=0.000001, help='learning rate')
parser.add_argument('--learning_rate_clip', type=float, default=0.0000001, help='learning rate clip')
parser.add_argument('--decay_rate', type=float, default=.9, help='decay rate for rmsprop')
parser.add_argument('--weight_decay', type=float, default=.0001, help='decay rate for rmsprop')
parser.add_argument('--batch_norm_decay', type=float, default=.999, help='decay rate for rmsprop')
parser.add_argument('--keep_prob', type=float, default=1.0, help='dropout keep probability')
parser.add_argument('--lamda_weights', type=float, default=0.01, help='weight of rotation error')
parser.add_argument('--data_argumentation', type=bool, default=True, help='whether do data argument')
parser.add_argument('--is_normalization', type=bool, default=True, help='whether do data normalization')
parser.add_argument('--target_image_size', default=[300, 300], nargs=2, type=int, help='Input images will be resized to this for data argumentation.')
#parser.add_argument('--output_dim', default=3, type=int, help='output dimention.')
#parser.add_argument('--feat_dim', default=128, type=int, help='feature dimention.')

'''Configure'''
parser.add_argument('--network', type=str, default='vggnet_localization')
parser.add_argument('--model_dir', type=str, default='/notebooks/global_localization/dual_resnet_torch', help='rnn, gru, or lstm')


parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/2012_01_08',
                                                            '/notebooks/michigan_nn_data/2012_01_15',
                                                            '/notebooks/michigan_nn_data/2012_01_22',
                                                            '/notebooks/michigan_nn_data/2012_02_02',
                                                            '/notebooks/michigan_nn_data/2012_02_04',
                                                            '/notebooks/michigan_nn_data/2012_02_05',
                                                            '/notebooks/michigan_nn_data/2012_03_31',
                                                            '/notebooks/michigan_nn_data/2012_09_28'])
'''
#parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/test'])
parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/2012_01_08'])
'''

parser.add_argument('--norm_tensor', type=str, default = ['/notebooks/global_localization/norm_mean_std.pt'])

parser.add_argument('--seed', default=1337, type=int)
parser.add_argument('--save_every', type=int, default=2000, help='save frequency')
parser.add_argument('--display', type=int, default=20, help='display frequency')
parser.add_argument('--tensorboard', type=bool, default=False, help='open tensorboard')
parser.add_argument('--train_validate_rate', type=float, default=0.7, help='split validation')

sys.argv = ['']
args = parser.parse_args()

if args.tensorboard:
    from torch.utils.tensorboard import SummaryWriter
    writer = SummaryWriter('runs/nn')

# Load Dataset

In [4]:
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import torchvision.transforms as transforms
import tf.transformations as tf_tran
from tqdm import tqdm
from PIL import Image
import numpy as np
import random

import gpytorch

import torch.nn as nn
import torch.optim as optim
from torchlib import resnet, vggnet
from torchlib.utils import LocalizationDataset,normalize
import time

transform = transforms.Compose([transforms.ToTensor()])
dataset = LocalizationDataset(dataset_dirs = args.train_dataset, \
                              image_size = args.target_image_size, \
                              transform = transform,
                              get_pair = False, sampling_rate=1)
'''
if len(args.train_dataset)>7:
    [args.norm_mean, args.norm_std] = [torch.tensor(x) for x in dataset.get_norm()]
    torch.save([args.norm_mean, args.norm_std], *args.norm_tensor)
    print('Save norm and std:',*args.norm_tensor)
else:
    [args.norm_mean, args.norm_std] = torch.load(*args.norm_tensor)
    print('Load norm and std:',*args.norm_tensor)
'''
[args.norm_mean, args.norm_std] = torch.load(*args.norm_tensor)
print('Load norm and std:',*args.norm_tensor)

dataloader = DataLoader(dataset, batch_size=args.batch_size, \
                        shuffle=True, num_workers=0, \
                        drop_last=False, pin_memory=True)
'''
lengths = [round(len(dataset)*args.train_validate_rate), round(len(dataset)*(1-args.train_validate_rate))]
training_dataset, val_dataset = random_split(dataset, lengths)

training_dataloader = DataLoader(training_dataset, batch_size=args.batch_size, \
                    shuffle=True, num_workers=0, \
                    drop_last=False, pin_memory=True)

val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, \
                    shuffle=True, num_workers=0, \
                    drop_last=False, pin_memory=True)
'''

100%|██████████| 28074/28074 [00:58<00:00, 479.00it/s]


Load norm and std: /notebooks/global_localization/norm_mean_std.pt


'\nlengths = [round(len(dataset)*args.train_validate_rate), round(len(dataset)*(1-args.train_validate_rate))]\ntraining_dataset, val_dataset = random_split(dataset, lengths)\n\ntraining_dataloader = DataLoader(training_dataset, batch_size=args.batch_size,                     shuffle=True, num_workers=0,                     drop_last=False, pin_memory=True)\n\nval_dataloader = DataLoader(val_dataset, batch_size=args.batch_size,                     shuffle=True, num_workers=0,                     drop_last=False, pin_memory=True)\n'

# Define Model

In [5]:
class Backbone(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = resnet.resnet50(pretrained=True)
    def forward(self,input_data):
        dense_feat = self.resnet(input_data)
        return dense_feat
    
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.global_context = vggnet.vggnet(input_channel=2048,opt="context")
        self.global_regressor = vggnet.vggnet(opt="regressor")
        
    def forward(self,input_data):
        context_feat = self.global_context(input_data)
        output,feature_t, feature_r = self.global_regressor(context_feat)
        return output, feature_t, feature_r
        
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = Backbone()
        self.nn = NN()
        
    def forward(self, *args):
        outputs = []
        for input_data in args:
            dense_feat = self.backbone(input_data)
            output, feature_t, feature_r = self.nn(dense_feat)
            outputs += [output]
        if len(args)>1:
            return outputs
        else:
            return output, feature_t, feature_r

In [6]:
class Trainer:
    def __init__(self, args, training = True):
        # data
        self.model = Model().cuda()
        self.args = args
        self.norm_mean = args.norm_mean.cuda()
        self.norm_std = args.norm_std.cuda()
        
        # training tool
        if training:
            self.optimizer = optim.Adam(self.model.parameters(), 
                                        lr=args.learning_rate, 
                                        weight_decay=args.weight_decay)
            self.scheduler = optim.lr_scheduler.LambdaLR(optimizer=self.optimizer,
                                                         lr_lambda=lambda epoch: args.decay_rate**epoch)
        
    def load_model(self, file_name = 'pretrained.pth', strict = True):
        # load file info
        state_dict = torch.load(os.path.join(self.args.model_dir, file_name))
        if 'net.resnet.conv1.weight' in state_dict:
            print('Transform from old model.')
            state_dict = self._from_old_model(state_dict)
            self.model.load_state_dict(state_dict,strict = strict)
        else:
            #print('Parameters layer:',len(state_dict.keys()))
            # load file to model
            self.model.load_state_dict(state_dict,strict = strict)
        #print("Successfully loaded model to {}.".format(self.device_name))
    
    def _from_old_model(self, state_dict, select = 'backbone'):
        for key in list(state_dict):
            if 'net.resnet.' in key:
                state_dict[key.replace('net.resnet.','backbone.resnet.')] = state_dict.pop(key)
            if 'net.global_regressor.' in key:
                state_dict[key.replace('net.global_regressor.','nn.global_regressor.')] = state_dict.pop(key)
            elif 'net.global_context.' in key:
                state_dict[key.replace('net.global_context.','nn.global_context.')] = state_dict.pop(key)
        return state_dict
         
    def save_model(self, file_name = 'model-{}-{}.pth'):
        checkpoint_path = os.path.join(args.model_dir, file_name)
        torch.save(self.model.state_dict(),checkpoint_path)
        print('Saving model to ' +  file_name)
            
    def train(self,x0, x1, y0, y1):
        # Step 0: zero grad
        self.optimizer.zero_grad()
        
        start = time.time()
        # Step 1: get data
        x0,x1,y0,y1 = x0.cuda(),x1.cuda(),y0.cuda(),y1.cuda()
        if args.is_normalization:
            y0, y1 = [normalize(y,self.norm_mean, self.norm_std) for y in [y0,y1]]
            
        # Step 2: training
        assert trainer.model.training == True
        single_loss = self._loss(x0, x1, y0, y1)
        batch_time = time.time() - start
        
        #Step 3: update
        single_loss.backward()
        self.optimizer.step()
        
        return float(single_loss), batch_time
            
    def _loss(self,x0, x1, y0, y1):
        # target relative
        relative_target_normed = get_relative_pose(y0, y1)
        # forward output
        global_output0,global_output1 = self.model(x0, x1)
        # output relative
        relative_consistence = get_relative_pose(global_output0,global_output1)
        
        # target loss
        global_loss = translational_rotational_loss(pred=global_output1, gt=y1, \
                                                    lamda=args.lamda_weights)
        # relative loss
        geometry_consistent_loss = translational_rotational_loss(pred=relative_consistence, \
                                                                 gt=relative_target_normed, \
                                                                 lamda=args.lamda_weights)
        total_loss = global_loss + geometry_consistent_loss        
        
        return total_loss
    
    def eval_forward(self,x,y,output_denormalize = True):
        # Step 1: get data
        x,y = x.cuda(),y.cuda()
        if args.is_normalization:
            y = normalize(y,self.norm_mean, self.norm_std)
        
        # Step 2: forward
        assert trainer.model.training == False
        output,_,_ = self.model(x)

        if args.is_normalization and output_denormalize:
            output = denormalize(output, self.norm_mean, self.norm_std)
            y = denormalize(y, self.norm_mean, self.norm_std)
            
        # Step 3: split output
        trans_target, rot_target = torch.split(y, [3, 4], dim=1)
        trans_prediction, rot_prediction = torch.split(output, [3, 4], dim=1)
        return trans_prediction, rot_prediction, trans_target, rot_target
    
    def dense_feature_forward(self,x):
        # Step 1: get data
        x = x.cuda()
        # Step 2: forward
        assert trainer.model.training == False
        dense_feat = self.model.backbone(x)
        dense_feat = self.model.nn.global_context(dense_feat)
        dense_feat = self.model.nn.global_regressor.regressor.flatten(dense_feat)
        return dense_feat

trainer = Trainer(args,training=True)
trainer.load_model('pretrained_old.pth', strict = False)

Transform from old model.


In [7]:
x_data = torch.zeros([len(dataset), 6400])
y_data = torch.zeros([len(dataset), 7])

# Extraction

In [8]:
trainer.model.eval()

for b, data in enumerate(dataloader, 0):
    start = time.time()
    x,y = data.values()
    
    with torch.no_grad():
        dense_feature = trainer.dense_feature_forward(x)
        if b == len(dataloader)-1:
            x_data[b*args.batch_size:] = dense_feature
            y_data[b*args.batch_size:] = y
        else:
            x_data[b*args.batch_size:(b+1)*args.batch_size] = dense_feature
            y_data[b*args.batch_size:(b+1)*args.batch_size] = y
            
    end = time.time()
    with torch.no_grad():
        if ((b+1)%5 == 0):
            print("{}/{}, time/batch = {:.3f}".format((b+1),len(dataloader),end - start))

5/641, time/batch = 0.998
10/641, time/batch = 0.996
15/641, time/batch = 1.005
20/641, time/batch = 1.000
25/641, time/batch = 1.004
30/641, time/batch = 1.009
35/641, time/batch = 1.010
40/641, time/batch = 1.008
45/641, time/batch = 1.015
50/641, time/batch = 1.012
55/641, time/batch = 1.024
60/641, time/batch = 1.015
65/641, time/batch = 1.009
70/641, time/batch = 1.013
75/641, time/batch = 1.014
80/641, time/batch = 1.013
85/641, time/batch = 1.013
90/641, time/batch = 1.024
95/641, time/batch = 1.023
100/641, time/batch = 1.017
105/641, time/batch = 1.014
110/641, time/batch = 1.012
115/641, time/batch = 1.016
120/641, time/batch = 1.015
125/641, time/batch = 1.018
130/641, time/batch = 1.016
135/641, time/batch = 1.017
140/641, time/batch = 1.013
145/641, time/batch = 1.017
150/641, time/batch = 1.019
155/641, time/batch = 1.012
160/641, time/batch = 1.013
165/641, time/batch = 1.017
170/641, time/batch = 1.019
175/641, time/batch = 1.017
180/641, time/batch = 1.019
185/641, tim

In [9]:
torch.save(x_data, 'x.pt')
torch.save(y_data, 'y.pt')